- ver_1 : アンサンブルのウェイトは最適化して実施。

# 注意点
- それぞれがtest に対する推測だけ行うこと。
- 自分のセクションの最後ではpred 以外全て削除。gc.collect()を二回走らせること。

In [ ]:

import pickle


# General
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import numpy as np
import os
import re
import os
import random
import gc
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import scipy

import torch
from tqdm.auto import tqdm        
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# calc_weight

In [ ]:
# kysky
kfsky_df = pd.read_csv("../input/toxic-kfsky-validaton-result/valid_kfsky.csv")

# makoto hori
#hori_df = pd.read_csv("../input/toxic-hori-validaton-result/valid_hori (1).csv")

# kma###
drop_cols = ["Unnamed: 0","cleaning_less","cleaning_more"]
kma_df = pd.read_csv("../input/kazuma-model5/kazuma_val.csv")
kma_df = kma_df.drop(drop_cols,axis = 1)
kma_df2= pd.read_csv("../input/kazuma-model5/valid_external.csv").loc[:,["preds_less_ext_kazuma","preds_more_ext_kazuma"]]
kma_df = pd.concat([kma_df,kma_df2],axis = 1)

# mogmog
mogmog_df = pd.read_csv("../input/validaton-result-morita/validation_df_add_pred.csv")

print(kfsky_df.shape)
#print(hori_df.shape)
print(kma_df.shape)
print(mogmog_df.shape)

# concat
concat_df = pd.concat([kfsky_df.iloc[:,1:],
                       #hori_df.iloc[:,3:],
                       kma_df.iloc[:,3:],
                       mogmog_df.iloc[:,3:]
                      ],axis = 1)

In [ ]:
# ユニークな文のみ取り出す。
less_cs = concat_df.columns[concat_df.columns.str.contains('less')]
more_cs = concat_df.columns[concat_df.columns.str.contains('more')]

columns_list = ['text'] + [f"model_{i}" for i in range(len(less_cs)-1)]
less_df = concat_df[less_cs]
more_df = concat_df[more_cs]
less_df.columns = columns_list
more_df.columns = columns_list
val_unique_df = pd.concat([less_df, more_df]).drop_duplicates('text').reset_index(drop=True)#.iloc[:,:-1]

In [ ]:
val_unique_df.head()

In [ ]:
# スケーリング揃える
scaler = MinMaxScaler()
tmp = scaler.fit_transform(val_unique_df.iloc[:,1:])
val_unique_df.iloc[:,1:] = tmp

In [ ]:
def scoring(pred):
    _val_unique_df = val_unique_df.copy()
    _val_unique_df["pred"] = pred
    val_unique_df_selected = _val_unique_df[["text","pred"]].set_index("text")["pred"].to_dict()
    
    concat_df["less_toxic_pred"] = concat_df["less_toxic"].map(val_unique_df_selected)
    concat_df["more_toxic_pred"] = concat_df["more_toxic"].map(val_unique_df_selected)
    concat_df["correct"] = (concat_df["less_toxic_pred"]  < concat_df["more_toxic_pred"]).astype(int)
    # スコア
    return concat_df["correct"].mean()

In [ ]:
num_models = val_unique_df.shape[1]-1
print(num_models)
def ensemble_score(weight_array): 
    avg_pred = np.sum([val_unique_df.loc[:,f"model_{i}"].values*weight_array[i] for i in range(num_models)],axis = 0)
    return -scoring(avg_pred)

In [ ]:
for i in range(1,num_models+1):
    l = concat_df.iloc[:,2*i]
    #print(l.head())
    m = concat_df.iloc[:,2*i+1]
    #print(m.head())
    print(np.mean((l< m).astype(int)))

In [ ]:
# 堀さんなしver
weight_array = np.array([1.0/num_models]*num_models)
weight_array[-1] = weight_array[-1]/50 # bertの重み初期値を小さくしておく

res = scipy.optimize.minimize(ensemble_score, weight_array, method='powell') 
print(res.x)
print(ensemble_score(res.x))

In [ ]:
del (kfsky_df, mogmog_df, kma_df, kma_df2, concat_df,less_df,more_df, val_unique_df)
gc.collect()
gc.collect()

- 432.8 MB

# kazuma section

In [ ]:
comments_df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

In [ ]:
def save_pkl(file_name, processor):
    OUTPUT_DIR = './'
    file_name = os.path.join(OUTPUT_DIR,file_name)
    pickle.dump(processor,open(file_name, 'wb'))
    print("FINISH")
def load_pkl(file_path):
    out_object = pickle.load(open(file_path, 'rb'))   
    return out_object
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
#     text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text
def ensemble(data, preds, cv_num):
    pred = np.zeros((data.shape[0],))

    for v in preds:
      pred += v

    return pred/cv_num

def multi_predict(test_x, models):
    preds = []
    for model in models:
        pred = model.predict(test_x)
        preds.append(pred)
    return preds

In [ ]:
# Dont need cleaning
ext_models = load_pkl('../input/kazuma-model5/ext_models.pkl')
ext_tfv = load_pkl('../input/kazuma-model5/ext_tfv.pkl')

# Need cleaning
tc_models = load_pkl('../input/kazuma-model5/kazuma_models_toxic.pkl')
tc_processor = load_pkl('../input/kazuma-model5/kazuma_processor.pkl')
rud_models = load_pkl('../input/kazuma-model5/kazuma_models_ruddit.pkl')

In [ ]:
tqdm.pandas()
comments_df['cleaning'] = comments_df['text'].progress_apply(text_cleaning)

In [ ]:
tfv_tc = tc_processor['toxc_comment_preprocessor']
tfv_rud = tc_processor['ruddit_preprocessor']

In [ ]:
tfv_col_comments = tfv_tc.transform(comments_df["cleaning"])
tfv_rud_col_comments = tfv_rud.transform(comments_df["cleaning"])
tfv_ext_col_comments = ext_tfv.transform(comments_df["text"]) # dont need cleaning

In [ ]:
preds = multi_predict(tfv_col_comments, tc_models)
preds_rud = multi_predict(tfv_rud_col_comments, rud_models)
preds_ext = multi_predict(tfv_ext_col_comments, ext_models)


preds_ens = ensemble(tfv_col_comments, preds, 3)
preds_rud_ens = ensemble(tfv_rud_col_comments, preds_rud, 3)
preds_ext_ens = ensemble(tfv_ext_col_comments, preds_ext, 3)

In [ ]:
sub = pd.DataFrame()
sub["comment_id"] = comments_df["comment_id"]
sub["kazuma_tc_score"] = preds_ens
sub["kazuma_rud_score"] = preds_rud_ens
sub["kazuma_ext_score"] = preds_ext_ens
sub

## gc

In [ ]:
del tfv_tc
del tfv_rud
del tfv_col_comments
del tfv_rud_col_comments
del tfv_ext_col_comments
del comments_df
del preds_ens
del preds_rud_ens
del preds_ext_ens
del tc_models
del rud_models
del ext_models
del ext_tfv
del tc_processor

gc.collect()
gc.collect()

- 511.7 MB

# fujii section

In [ ]:
from tqdm import tqdm
from time import time
from contextlib import contextmanager
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline
import re

import fasttext
import fasttext.util

import os
import pickle

import warnings
warnings.simplefilter('ignore')

In [ ]:
train_df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
test_df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
valid_df = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")

In [ ]:
train_text = train_df[["id", "comment_text"]]

valid_text_l = pd.DataFrame()
valid_text_l["comment_text"] = valid_df["less_toxic"]
valid_text_l["id"] = "less"

valid_text_m = pd.DataFrame()
valid_text_m["comment_text"] = valid_df["more_toxic"]
valid_text_m["id"] = "more"

valid_text = pd.concat([valid_text_m, valid_text_l],axis=0)

check_df = pd.concat([train_text, valid_text], axis=0)

In [ ]:
# drop_id
drop_id = check_df[(check_df["comment_text"].duplicated(keep=False))&(check_df["id"]!="less")&(check_df["id"]!="more")]["id"].unique()
train_df = train_df[~train_df["id"].isin(drop_id)]

In [ ]:
# check
from matplotlib_venn import venn2
venn2([set(train_df['comment_text']), set(valid_df['more_toxic'])], set_labels=('train', 'test_more'));

In [ ]:
venn2([set(train_df['comment_text']), set(valid_df['less_toxic'])], set_labels=('train', 'test_less'));

In [ ]:
del check_df
del train_text
del valid_text
del valid_text_l
del valid_text_m
gc.collect()
gc.collect()

In [ ]:
# 学習データのスコアを決定する➟これは調整とのこと
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    train_df[category] = train_df[category] * cat_mtpl[category]
    
train_df["score"] = train_df.loc[:, "toxic": "identity_hate"].sum(axis=1)
train_df["y"] = train_df["score"]

min_len = (train_df["y"]>=0.1).sum()
y0_undersample = train_df[train_df["y"]==0].sample(n=min_len, random_state=2021)
new_train_df = pd.concat([train_df[train_df["y"]>=0.1], y0_undersample])

new_train_df = new_train_df[["id", "comment_text", "y"]].reset_index(drop=True)

In [ ]:
del train_df
gc.collect()
gc.collect()

In [ ]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text


class BaseBlock(object):
    def fit(self, input_df, y=None):
        return self.transform(input_df)
    
    def transform(self, input_df):
        raise NotImplementedError()

        
##max_featuresを調整することでスコアが改善する可能性
class TfidfBlock(BaseBlock):
    def __init__(self, column: str, whole_df: pd.DataFrame, decomposition: str, n_compose: int):
        self.column = column
        self.whole_df = whole_df
        self.decomposition = decomposition
        self.n_compose = n_compose

    def fit(self, input_df, y=None):
        master_df = self.whole_df
        text = self.whole_df[self.column].fillna("")

        if self.decomposition == "svd":
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("svd", TruncatedSVD(n_components=self.n_compose, random_state=71))
            ])

        elif self.decomposition == "NMF":
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("NMF", NMF(n_components=self.n_compose, random_state=71))
            ])
        elif self.decomposition == "LDA":
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("NMF", LatentDirichletAllocation(n_components=self.n_compose, random_state=71))
            ])
        else:
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("svd", TruncatedSVD(n_components=self.n_compose, random_state=71))
            ])

        self.pipeline_.fit(text)

        return self.transform(input_df)

    def transform(self, input_df):
        text = input_df[self.column].fillna("")
        z = self.pipeline_.transform(text)

        out_df = pd.DataFrame(z)
        return out_df.add_prefix(f'{self.column}_tfidf_{self.decomposition}_')
    

class StringLengthBlock(BaseBlock):
    def __init__(self, column):
        self.column = column

    def transform(self, input_df):
        output_df = pd.DataFrame()
        output_df[self.column] = input_df[self.column].str.len()
        return output_df.add_prefix("StringLength_")

    
class CountVectorizerBlock(BaseBlock):
    def __init__(self, column: str, whole_df: pd.DataFrame, decomposition: str, n_compose: int):
        self.column = column
        self.whole_df = whole_df
        self.decomposition = decomposition
        self.n_compose = n_compose

    def fit(self, input_df, y=None):
        master_df = self.whole_df
        text = self.whole_df[self.column].fillna("")

        if self.decomposition == "svd":
            self.pipeline_ = Pipeline([
                ("tfidf", CountVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("svd", TruncatedSVD(n_components=self.n_compose, random_state=71))
            ])

        elif self.decomposition == "NMF":
            self.pipeline_ = Pipeline([
                ("tfidf", CountVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("NMF", NMF(n_components=self.n_compose, random_state=71))
            ])
        elif self.decomposition == "LDA":
            self.pipeline_ = Pipeline([
                ("tfidf", CountVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("NMF", LatentDirichletAllocation(n_components=self.n_compose, random_state=71))
            ])
        else:
            self.pipeline_ = Pipeline([
                ("tfidf", CountVectorizer(min_df= 3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5), max_features=50000)),
                ("svd", TruncatedSVD(n_components=self.n_compose, random_state=71))
            ])

        self.pipeline_.fit(text)

        return self.transform(input_df)

    def transform(self, input_df):
        text = input_df[self.column].fillna("")
        z = self.pipeline_.transform(text)

        out_df = pd.DataFrame(z)
        return out_df.add_prefix(f'{self.column}_CountVectorizer_{self.decomposition}_')

    
class WordCountBlock(BaseBlock):
    def __init__(self, column):
        self.column = column

    def transform(self, input_df):
        output_df = pd.DataFrame()
        output_df[self.column] = input_df[self.column].astype(str).map(lambda x: len(x.split()))
        return output_df.add_prefix("WordCount_")
    

## FastText

# 今回使用せず
def get_text_series(input_df: pd.DataFrame, column: str, sep='&'):
    out_series = None
    for i, c in enumerate(column.split(sep)):
        text_i = input_df[c].astype(str)
        if out_series is None:
            out_series = text_i
        else:
            out_series = out_series + ' ' + text_i
    return out_series

# 今回使用せず
def create_embedding(document: str, model):
    words = document.split(" ")
    x = [model.get_word_vector(w) for w in words]
    x = np.max(x, axis=0)
    return x


def load_fasttext_model():
    ft = fasttext.load_model("../input/fast100/cc.en.100.bin")
    ft = fasttext.util.reduce_model(ft, 100)

    return ft


class FasttextEmbeddingBlock(BaseBlock):
    def __init__(self, column: str):
        self.column = column

    def fit(self, input_df, y=None, **kwargs):
        self.ft = load_fasttext_model()

        return self.transform(input_df)

    def transform(self, input_df):
        # この書き方知らなかった。
        emb = np.stack(input_df['comment_text'].map(lambda x: self.ft.get_sentence_vector(x)).values)
        output_df = pd.DataFrame(emb)
        return output_df.add_prefix(f'{self.column}_FastText')

In [ ]:
@contextmanager
def timer(logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None):
    if prefix: format_str = str(prefix) + format_str
    if suffix: format_str = format_str + str(suffix)
    start = time()
    yield
    d = time() - start
    out_str = format_str.format(d)
    if logger:
        logger.info(out_str)
    else:
        print(out_str)


def get_function(block, is_train):
    s = mapping = {
        True: 'fit',
        False: 'transform'
    }.get(is_train)
    return getattr(block, s)


def to_feature(input_df,
               blocks,
               is_train=False):
    out_df = pd.DataFrame()

    for block in tqdm(blocks, total=len(blocks)):
        func = get_function(block, is_train)

        with timer(prefix='create ' + str(block) + ' '):
            _df = func(input_df)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
    return reduce_mem_usage(out_df)

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
              .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
test_df.rename(columns = {"text": "comment_text"}, inplace=True)

In [ ]:
tqdm.pandas()
new_train_df['comment_text'] = new_train_df['comment_text'].progress_apply(text_cleaning)
test_df['comment_text'] = test_df['comment_text'].progress_apply(text_cleaning)

In [ ]:
process_blocks = [
    FasttextEmbeddingBlock("comment_text"),
    TfidfBlock("comment_text" ,whole_df=new_train_df, decomposition="svd", n_compose=500),
    CountVectorizerBlock("comment_text" ,whole_df=new_train_df, decomposition="svd", n_compose=500),
    StringLengthBlock("comment_text"),
    WordCountBlock("comment_text")
]


train_x = to_feature(new_train_df, process_blocks, is_train=True)
test_x = to_feature(test_df, process_blocks)

- 13GB超える。

最初から100次元に圧縮したbinary呼ぶことで対応

In [ ]:
# predict lightGBM
model_lightgbm = []
for i in range(5):
    model = pickle.load(open(f"../input/kfsky-model1/lightGBM_kfsky_fold{i}", 'rb'))
    model_lightgbm.append(model)

pred_lightgbm = np.array([model.predict(test_x.values) for model in model_lightgbm])
pred_lightgbm = np.mean(pred_lightgbm, axis=0)

In [ ]:
vec = pickle.load(open("../input/kfsky-model1/Ridge_tfidf", 'rb'))
#X = vec.fit_transform(new_train_df['comment_text'])
X_test = vec.transform(test_df['comment_text'])

In [ ]:
ridge_model = pickle.load(open("../input/kfsky-model1/Ridge_kfsky", 'rb'))
pred_ridge = ridge_model.predict(X_test)

In [ ]:
pred_ridge

In [ ]:
sub_kfsky = pd.DataFrame()
sub_kfsky["comment_id"] = test_df["comment_id"]
sub_kfsky["kfsky_lightgbm"] = pred_lightgbm
sub_kfsky["kfsky_ridge"] = pred_ridge
sub_kfsky.head()

In [ ]:
del new_train_df
del test_df
del train_x
del test_x
del X_test
del vec
del model_lightgbm
del pred_lightgbm
del ridge_model
del pred_ridge
del process_blocks
gc.collect()
gc.collect()

# mogmog section

In [ ]:
class CFG:
    debug=False
    seed=42
    n_fold = 4
    model_name = "../input/roberta-base-edited"
    max_len = 256
    text="text_clean2"
    target="target"
    target_size = 1
    hidden_size = 768
    fc_dropout = 0.
    batch_size = 32
    num_workers = 4
    model_dir = "../input/roberta-001-train-jigsaw1"

In [ ]:
import os
import gc
import re
import time
import math
import random
import string
from collections import Counter
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

from gensim import models
from gensim.models import KeyedVectors,FastText

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, mutual_info_regression,chi2, f_regression
from bs4 import BeautifulSoup

from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from scipy import sparse
import scipy
import seaborn as sns
from unicodedata import category, name, normalize

import lightgbm as lgb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR

# NLP
from transformers import AutoTokenizer, AutoModel,get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import pickle
import warnings
warnings.filterwarnings('ignore')

pd.set_option("max_columns",100)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
# 余計なスペースの削除
spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad', '\xa0']
def remove_space(text):
    """
    remove extra spaces and ending space if any
    """
    for space in spaces:
        text = text.replace(space, ' ')
    text = text.strip()
    text = re.sub('\s+', ' ', text)
    return text


special_punc_mappings = {"—": "-", "–": "-", "_": "-", '”': '"', "″": '"', '“': '"', '•': '.', '−': '-',
                         "’": "'", "‘": "'", "´": "'", "`": "'", '\u200b': ' ', '\xa0': ' ','،':'','„':'',
                         '…': ' ... ', '\ufeff': ''}
def clean_special_punctuations(text):
    for punc in special_punc_mappings:
        if punc in text:
            text = text.replace(punc, special_punc_mappings[punc])
    #text = remove_diacritics(text)
    return text

# 数字
def clean_number(text):
    # 正直数字全消しでもいい気がするけど...
    text = re.sub(r'(\d+)([a-zA-Z])', '\g<1> \g<2>', text)
    text = re.sub(r'(\d+) (th|st|nd|rd) ', '\g<1>\g<2> ', text)
    text = re.sub(r'(\d+),(\d+)', '\g<1>\g<2>', text)
    
    return text


# nbのまま持ってきたけど、意味ない場合もあり。
rare_words_mapping = {' s.p ': ' ', ' S.P ': ' ', 'U.s.p': '', 'U.S.A.': 'USA', 'u.s.a.': 'USA', 'U.S.A': 'USA',
                      'u.s.a': 'USA', 'U.S.': 'USA', 'u.s.': 'USA', ' U.S ': ' USA ', ' u.s ': ' USA ', 'U.s.': 'USA',
                      ' U.s ': 'USA', ' u.S ': ' USA ', 'fu.k': 'fuck', 'U.K.': 'UK', ' u.k ': ' UK ',
                      ' don t ': ' do not ', 'bacteries': 'batteries', ' yr old ': ' years old ', 'Ph.D': 'PhD',
                      'cau.sing': 'causing', 'Kim Jong-Un': 'The president of North Korea', 'savegely': 'savagely',
                      'Ra apist': 'Rapist', '2fifth': 'twenty fifth', '2third': 'twenty third',
                      '2nineth': 'twenty nineth', '2fourth': 'twenty fourth', '#metoo': 'MeToo',
                      'Trumpcare': 'Trump health care system', '4fifth': 'forty fifth', 'Remainers': 'remainder',
                      'Terroristan': 'terrorist', 'antibrahmin': 'anti brahmin',
                      'fuckboys': 'fuckboy', 'Fuckboys': 'fuckboy', 'Fuckboy': 'fuckboy', 'fuckgirls': 'fuck girls',
                      'fuckgirl': 'fuck girl', 'Trumpsters': 'Trump supporters', '4sixth': 'forty sixth',
                      'culturr': 'culture',
                      'weatern': 'western', '4fourth': 'forty fourth', 'emiratis': 'emirates', 'trumpers': 'Trumpster',
                      'indans': 'indians', 'mastuburate': 'masturbate', 'f**k': 'fuck', 'F**k': 'fuck', 'F**K': 'fuck',
                      ' u r ': ' you are ', ' u ': ' you ', '操你妈': 'fuck your mother', 'e.g.': 'for example',
                      'i.e.': 'in other words', '...': '.', 'et.al': 'elsewhere', 'anti-Semitic': 'anti-semitic',
                      'f***': 'fuck', 'f**': 'fuc', 'F***': 'fuck', 'F**': 'fuc','f*ck':'fuck',
                      'a****': 'assho', 'a**': 'ass', 'h***': 'hole', 'A****': 'assho', 'A**': 'ass', 'H***': 'hole',
                      's***': 'shit', 's**': 'shi', 'S***': 'shit', 'S**': 'shi', 'Sh**': 'shit','sh*t':'shit',
                      'p****': 'pussy', 'p*ssy': 'pussy', 'P****': 'pussy',
                      'p***': 'porn', 'p*rn': 'porn', 'P***': 'porn',
                      'st*up*id': 'stupid',
                      'd***': 'dick', 'di**': 'dick', 'h*ck': 'hack',
                      'b*tch': 'bitch', 'bi*ch': 'bitch', 'bit*h': 'bitch', 'bitc*': 'bitch', 'b****': 'bitch',
                      'b***': 'bitc', 'b**': 'bit', 'b*ll': 'bull'
                      }

def pre_clean_rare_words(text):
    for rare_word in rare_words_mapping:
        if rare_word in text:
            text = text.replace(rare_word, rare_words_mapping[rare_word])
    return text



# de-contract the contraction
def decontracted(text):
    # specific
    text = re.sub(r"(W|w)on(\'|\’)t ", "will not ", text)
    text = re.sub(r"(C|c)an(\'|\’)t ", "can not ", text)
    text = re.sub(r"(Y|y)(\'|\’)all ", "you all ", text)
    text = re.sub(r"(Y|y)a(\'|\’)ll ", "you all ", text)

    # general
    text = re.sub(r"(I|i)(\'|\’)m ", "i am ", text)
    text = re.sub(r"(A|a)in(\'|\’)t ", "is not ", text)
    text = re.sub(r"n(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)re ", " are ", text)
    text = re.sub(r"(\'|\’)s ", " is ", text)
    text = re.sub(r"(\'|\’)d ", " would ", text)
    text = re.sub(r"(\'|\’)ll ", " will ", text)
    text = re.sub(r"(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)ve ", " have ", text)
    return text



regular_punct = list(string.punctuation)
extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤']
all_punct = list(set(regular_punct + extra_punct))
# do not spacing - and .
all_punct.remove('-')
all_punct.remove('.')
#all_punct.remove('!') # !の連続は重要な気がするので、あえて間を開けなくてもいいかな？？？

def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text
def remove_punctuation(text):
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, ' ')
    return text



mis_connect_list = ['(W|w)hat', '(W|w)hy', '(H|h)ow', '(W|w)hich', '(W|w)here', '(W|w)ill']
mis_connect_re = re.compile('(%s)' % '|'.join(mis_connect_list))

mis_spell_mapping = {'whattsup': 'WhatsApp', 'whatasapp':'WhatsApp', 'whatsupp':'WhatsApp', 
                      'whatcus':'what cause', 'arewhatsapp': 'are WhatsApp', 'Hwhat':'what',
                      'Whwhat': 'What', 'whatshapp':'WhatsApp', 'howhat':'how that',
                      # why
                      'Whybis':'Why is', 
                      # How
                      "Howddo":"How do", 'Howeber':'However'}
def spacing_some_connect_words(text):
    """
    'Whyare' -> 'Why are'
    """
    ori = text
    for error in mis_spell_mapping:
        if error in text:
            text = text.replace(error, mis_spell_mapping[error])
            
    # what
    text = re.sub(r" (W|w)hat+(s)*[A|a]*(p)+ ", " WhatsApp ", text)
    text = re.sub(r" (W|w)hat\S ", " What ", text)
    text = re.sub(r" \S(W|w)hat ", " What ", text)
    # why
    text = re.sub(r" (W|w)hy\S ", " Why ", text)
    text = re.sub(r" \S(W|w)hy ", " Why ", text)
    # How
    text = re.sub(r" (H|h)ow\S ", " How ", text)
    text = re.sub(r" \S(H|h)ow ", " How ", text)
    # which
    text = re.sub(r" (W|w)hich\S ", " Which ", text)
    text = re.sub(r" \S(W|w)hich ", " Which ", text)
    # where
    text = re.sub(r" (W|w)here\S ", " Where ", text)
    text = re.sub(r" \S(W|w)here ", " Where ", text)
    # 
    text = mis_connect_re.sub(r" \1 ", text)
    text = text.replace("What sApp", 'WhatsApp')
    
    text = remove_space(text)
    return text



def remove_number(text):
    text = re.sub(r'[0-9]+', '', text)
    return text


def get_lower(text):
    text = text.lower()
    return text

In [ ]:
# https://www.kaggle.com/manabendrarout/pytorch-roberta-ranking-baseline-jrstc-train/notebook
def text_cleaning_1(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
        ↑これは実施せず。多分しない方がマシなような気がする。たぶん.
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # 引用nbより
    text = remove_space(text)
    text = clean_special_punctuations(text)
    text = clean_number(text)
    text = pre_clean_rare_words(text)
    text = decontracted(text)
    text = spacing_punctuation(text)
    text = spacing_some_connect_words(text)
    text = remove_space(text)
    
    # 元々関数
    #text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    #text = re.sub(' +', ' ', text) #Remove Extra Spaces
    #text = text.strip() # remove spaces at the beginning and at the end of string

    return text

def text_cleaning_2(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
        ↑これは実施せず。多分しない方がマシなような気がする。たぶん.
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text



# https://www.kaggle.com/manabendrarout/pytorch-roberta-ranking-baseline-jrstc-train/notebook
def text_cleaning_3(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
        ↑これは実施せず。多分しない方がマシなような気がする。たぶん.
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # 元々関数
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = remove_number(text) #数字も消す
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string
    text = get_lower(text)

    return text


In [ ]:
def make_basic_feat(input_df):
    out_df = pd.DataFrame()
    # 文字数
    out_df["Num_character"] = input_df["text"].apply(lambda x:len(x))
    # 単語数
    out_df["Num_word"] = input_df["text"].apply(lambda x:len(x.split()))
    
    return out_df

def get_gensim_embed(texts,ndim):
    # 埋め込み用のarray
    swem_embedding = np.zeros((len(texts), ndim))
    
    # 埋め込み取得
    for i, text in enumerate(tqdm(texts)):
        embeddings = [w2v_model.get_vector(word)
                      if w2v_model.key_to_index.get(word) is not None
                      else np.zeros(ndim, dtype=np.float32)
                      for word in text.split()
                     ]
        if len(embeddings) > 0:
            mean_vector = np.mean(np.stack(embeddings), axis=0)
            swem_embedding[i] = mean_vector
    return swem_embedding

def get_fasttext_embed(texts,ndim):
    # 埋め込み用のarray
    swem_embedding = np.zeros((len(texts), ndim))
    
    # 埋め込み取得
    for i, text in enumerate(tqdm(texts)):
        tokens = [word for word in text.split()]
        if len(tokens)>0:
            mean_vector = np.mean(fmodel.wv[tokens], axis = 0)
            #print(mean_vector.shape)
            swem_embedding[i] = mean_vector
    return swem_embedding

In [ ]:
use_cols = [
    "text_clean1",
    "text_clean2",
    "text_clean3"
]
def get_val_tfidf_dict(vec_dict, val_df_for_pred):
    out_dict = {}
    for _name,_vec in tqdm(vec_dict.items()):
        _col = "_".join(_name.split("_")[-2:]) 
        out_dict[_name] = _vec.transform(val_df_for_pred[_col])
    return out_dict

def get_val_select_topfeat_and_svd(tfidf_df_val_dict,selector_dict,svd_transfomer_dict):
    svd_matrix_list = []
    for i in tqdm(range(6)):
        _mat = list(tfidf_df_val_dict.values())[i]
        _selector = list(selector_dict.values())[i]
        _svd_transformer = list(svd_transfomer_dict.values())[i]

        _matrix_filtered = _selector.transform(_mat)
        _svd_matrix = _svd_transformer.transform(_matrix_filtered)
        svd_matrix_list.append(_svd_matrix)

    val_svd_matrix =  np.hstack(svd_matrix_list)
    return val_svd_matrix

In [ ]:
def valid(model_dict, df_val_dict):
    preds_list = []
    for _name,_model_list in model_dict.items():
        _preds =[]
        for i in range(5):
            pred = _model_list[i].predict(df_val_dict[_name])
            _preds.append(pred)
        preds_list.append(np.mean(_preds, axis=0))
    return preds_list

def valid2(lgb_model_dict):
    preds_list = []
    for i in tqdm(range(3)):
        _gen = list(gensim_df_val_dict.values())[i]
        _fast = list(fasttext_df_val_dict.values())[i]

        mat = np.hstack([_gen,
                         _fast,
                         basic_feat_val_df[use_basic_feat].values
                        ])

        _preds =[]
        _model_list = list(lgb_model_dict.values())[i]

        for j in range(5):
            pred = _model_list[j].predict(mat)
            _preds.append(pred)
        preds_list.append(np.mean(_preds, axis=0))
    return preds_list

In [ ]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

class JigsawDataset(Dataset):
    def __init__(self, CFG, input_df, is_train=True):
        self.CFG = CFG
        self.is_train = is_train # 推論時のlabelがない時はFalseにしておく。
        self.text = input_df[self.CFG.text].values
        self.tokenizer = AutoTokenizer.from_pretrained(self.CFG.model_name)
        if self.is_train:
            self.labels = input_df[self.CFG.target].values       
             
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        text =  self.text[idx]
        encoded = self.tokenizer.encode_plus(
            text,
            truncation=True,# 最大長で切り捨てを行う。
            add_special_tokens=True,#「スペシャルトークン」は[CLS][SEP]みたいなやつ
            max_length=self.CFG.max_len,
            padding='max_length'
        )
        input_ids = torch.tensor(encoded['input_ids'])
        attention_mask = torch.tensor(encoded['attention_mask'])
        
        if self.is_train:
            #label = torch.tensor(self.labels[idx]).float()
            label = torch.tensor(self.labels[idx])
            return input_ids, attention_mask, label
        return input_ids, attention_mask
    
class JigsawModel(nn.Module):
    '''
    あとで隠れ層取り出せるような構造にしておく(可視化or別のモデルに突っ込みたい)。カズマに教えてもらえるかも。
    - https://www.kaggle.com/yasufuminakama/jigsaw4-luke-base-starter-train/notebook
    - https://qiita.com/niship2/items/f84751aed893da869cec
    '''
    def __init__(self, CFG):
        super().__init__()
        self.CFG = CFG
        self.model = AutoModel.from_pretrained(self.CFG.model_name)
        self.fc_dropout = nn.Dropout(self.CFG.fc_dropout)
        self.fc = nn.Linear(self.CFG.hidden_size, self.CFG.target_size)
    
    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids = input_ids, 
                         attention_mask = attention_mask)
        # outのshapeは[batch, 系列長, 隠れ層次元]　。文章の頭には[CLS]トークンがいるので、それを持ってきている?? あとでちゃんと理解して、工夫する必要あれば実施。
        # https://www.kaggle.com/debarshichanda/pytorch-w-b-jigsaw-starter
        # これはいろんな考え方があるので、いじれるはず。
        # https://www.kaggle.com/yasufuminakama/jigsaw4-luke-base-starter-train/notebook
        
        out = self.fc_dropout(out[1])
        outputs = self.fc(out)
        return outputs

def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    
    for step, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total = len(test_loader)):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask)
        preds.append(y_preds.to("cpu").numpy())
        #preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
def load_pkl(file_path):
    out_object = pickle.load(open(file_path, 'rb'))   
    return out_object

In [ ]:
# pklファイルの読み込み
tfidf_vec_dict = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/tfidf_vec_dict.pkl")
tfidf_vec_dict_ruddit = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/tfidf_vec_dict_ruddit.pkl")
tfidf_feat_selector = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/tfidf_feat_selector.pkl")
tfidf_svd_transfomer = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/tfidf_svd_transfomer.pkl")
tfidf_feat_selector_ruddit = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/tfidf_feat_selector_ruddit.pkl")
tfidf_svd_transfomer_ruddit = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/tfidf_svd_transfomer_ruddit.pkl")

ridge_model_dict_1 = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/ridge_model_dict_1.pkl")
ridge_model_dict_1_ruddit = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/ridge_model_dict_1_ruddit.pkl")
lgb_model_dict_2 = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/lgb_model_dict_2.pkl")
lgb_model_dict_1_ruddit = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/lgb_model_dict_1_ruddit.pkl")
lgb_model_dict_3 = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/lgb_model_dict_3.pkl")
lgb_model_dict_2_ruddit = load_pkl("../input/jigsaw1st-ruddit-ensemble-addbert-001-savepkl/lgb_model_dict_2_ruddit.pkl")

In [ ]:
test_df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

# クリーニング
tqdm.pandas()
test_df["text_clean1"] = test_df['text'].progress_apply(text_cleaning_1)
test_df["text_clean2"] = test_df['text'].progress_apply(text_cleaning_2)
test_df["text_clean3"] = test_df['text'].progress_apply(text_cleaning_3)

In [ ]:
tfidf_df_test_dict = get_val_tfidf_dict(tfidf_vec_dict, test_df)
tfidf_df_test_dict_ruddit = get_val_tfidf_dict(tfidf_vec_dict_ruddit, test_df)

test_svd_matrix = get_val_select_topfeat_and_svd(tfidf_df_test_dict,
                                                tfidf_feat_selector,
                                                tfidf_svd_transfomer)
test_svd_matrix_ruddit = get_val_select_topfeat_and_svd(tfidf_df_test_dict_ruddit,
                                                       tfidf_feat_selector_ruddit,
                                                       tfidf_svd_transfomer_ruddit)

In [ ]:
w2v_model = KeyedVectors.load("../input/gensim-googlenewsvectorsnegative300/GoogleNews-vectors-negative300.gensim")
gensim_df_test_dict = {}
for _col in tqdm(use_cols):
    gensim_df_test_dict[f"gensim_{_col}"] = get_gensim_embed(test_df[_col],ndim=300)
    
del w2v_model
gc.collect()

fmodel = FastText.load('../input/jigsaw-regression-based-data/FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin')
fasttext_df_test_dict = {}
for _col in tqdm(use_cols):
    fasttext_df_test_dict[f"fasttext_{_col}"] = get_fasttext_embed(test_df[_col],ndim=256)

del fmodel
gc.collect()

basic_feat_test_df = make_basic_feat(test_df)

### 推論

In [ ]:
_preds_list1 = valid(ridge_model_dict_1, tfidf_df_test_dict)
pred1 = np.mean(_preds_list1,axis = 0)

_preds_list2 = valid(ridge_model_dict_1_ruddit, tfidf_df_test_dict_ruddit)
pred2 = np.mean(_preds_list2,axis = 0)

In [ ]:
use_basic_feat = ["Num_character", "Num_word"]
preds_list = []
preds_list_ruddit = []
for i in range(3):
    _gen = list(gensim_df_test_dict.values())[i]
    _fast = list(fasttext_df_test_dict.values())[i]
    
    mat = np.hstack([_gen,
                     _fast,
                     basic_feat_test_df[use_basic_feat].values
                    ])
    
    _preds =[]
    _preds_ruddit =[]
    
    _model_list = list(lgb_model_dict_2.values())[i] 
    _model_list_ruddit = list(lgb_model_dict_1_ruddit.values())[i] 
    
    for j in range(5):
        pred = _model_list[j].predict(mat)
        pred_ruddit = _model_list_ruddit[j].predict(mat)
        _preds.append(pred)
        _preds_ruddit.append(pred_ruddit)
        
    preds_list.append(np.mean(_preds, axis=0))
    preds_list_ruddit.append(np.mean(_preds_ruddit, axis=0))

pred3 = np.mean(preds_list,axis = 0)
pred4 = np.mean(preds_list_ruddit,axis = 0)

In [ ]:
preds5 = []
for _model in lgb_model_dict_3['TF-IDF(SVD)_600dim']:
    pred = _model.predict(test_svd_matrix)
    preds5.append(pred)
pred5 = np.mean(preds5,axis = 0)

preds6 = []
for _model in lgb_model_dict_2_ruddit['TF-IDF(SVD)_600dim']:
    pred = _model.predict(test_svd_matrix_ruddit)
    preds6.append(pred)
pred6 = np.mean(preds6,axis = 0)

In [ ]:
test_dataset = JigsawDataset(CFG, test_df, is_train = False)
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, 
                         pin_memory=True, 
                         drop_last=False)

In [ ]:
preds7 = []
for fold in range(CFG.n_fold):
    model = JigsawModel(CFG)
    state = torch.load(CFG.model_dir+f"/{CFG.model_name.split('/')[-1]}_fold{fold}_best.pth", map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    preds7.append(prediction)
    del model, state; gc.collect()
    torch.cuda.empty_cache()
pred7 = np.mean(preds7,axis = 0)

In [ ]:
del test_dataset, test_loader
gc.collect()

In [ ]:
scaler = MinMaxScaler()
pr1_scaled = scaler.fit_transform(pred1.reshape(-1,1))
pr2_scaled = scaler.fit_transform(pred2.reshape(-1,1))
pr3_scaled = scaler.fit_transform(pred3.reshape(-1,1))
pr4_scaled = scaler.fit_transform(pred4.reshape(-1,1))
pr5_scaled = scaler.fit_transform(pred5.reshape(-1,1))
pr6_scaled = scaler.fit_transform(pred6.reshape(-1,1))
pr7_scaled = scaler.fit_transform(pred7.reshape(-1,1))

pred_scaled_mat_test = np.hstack([pr1_scaled,
                                  pr2_scaled,
                                  pr3_scaled,
                                  pr4_scaled,
                                  pr5_scaled,
                                  pr6_scaled,
                                  pr7_scaled
                            ])
pred_scaled_mat_test = pd.DataFrame(pred_scaled_mat_test,
                                    columns = ["ridge","ridge_rud","lgb_emb","lgb_emb_rud","lgb_svd","lgb_svd_rud","roberta"])
pred_scaled_mat_test = pred_scaled_mat_test.add_prefix("mogmog_")

In [ ]:
mogmog_sub = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv').drop("score",axis = 1)
mogmog_sub = pd.concat([mogmog_sub,pred_scaled_mat_test],axis = 1)

In [ ]:
mogmog_sub.head()

In [ ]:
del (tfidf_vec_dict,
     tfidf_vec_dict_ruddit,
     tfidf_feat_selector,
     tfidf_svd_transfomer,
     tfidf_feat_selector_ruddit,
     tfidf_svd_transfomer_ruddit,
     ridge_model_dict_1,
     ridge_model_dict_1_ruddit,
     lgb_model_dict_2,
     lgb_model_dict_1_ruddit,
     lgb_model_dict_3,
     lgb_model_dict_2_ruddit,
     pred_scaled_mat_test,
     tfidf_df_test_dict,
     tfidf_df_test_dict_ruddit,
     test_svd_matrix,
     test_svd_matrix_ruddit,
     gensim_df_test_dict,
     fasttext_df_test_dict
    )
gc.collect()
gc.collect()

- あとは重みres.xをかけてやればok。

# アンサンブル
- 最適化した重みでサブミット

In [ ]:
all_sub_df = pd.concat([
    sub_kfsky,
    sub.iloc[:,1:],#kma
    mogmog_sub.iloc[:,1:]#mogmog
],axis = 1)

all_sub_df.head()

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(20,12))
sns.heatmap(all_sub_df.corr(),square = True,annot = True, fmt=".1f",ax = axes[0], vmax = 1, vmin =-1,cmap = "RdBu")
axes[0].set_title("corr")
sns.heatmap(all_sub_df.corr(method = "kendall"),square = True,annot = True, fmt=".1f",ax = axes[1], vmax = 1, vmin = -1,cmap = "RdBu")
axes[1].set_title("corr(kendall)")

- kfsky ridgeが他のモデルと大きく違う

In [ ]:
scaler = MinMaxScaler()
all_sub_df.iloc[:,1:] = pd.DataFrame(scaler.fit_transform(all_sub_df.iloc[:,1:]))

all_sub_df.describe()

In [ ]:
all_sub_df["score"] = np.sum(all_sub_df.iloc[:,1:]*res.x,axis = 1)

In [ ]:
all_sub_df["score"].hist(bins = 100)

In [ ]:
all_sub_df['score'] = all_sub_df['score'].rank(method='first')
all_sub_df[['comment_id', 'score']].to_csv('submission.csv', index=False)

## eqauly_weight
- CVの最適化が怪しいので、各自のモデルを単純平均して一人1つの結果にした後で、最後単純平均アンサンブルする.

In [ ]:
# def scale_and_averaging(df):
#     scaler = MinMaxScaler()
#     df.iloc[:,1:] = pd.DataFrame(scaler.fit_transform(df.iloc[:,1:]))
#     return df.iloc[:,1:].mean(axis = 1)

In [ ]:
# kfsky_pred_final = scale_and_averaging(sub_kfsky)
# kma_pred_final = scale_and_averaging(sub)
# mogmog_pred_final = scale_and_averaging(mogmog_sub)

In [ ]:
# plt.hist(kfsky_pred_final,alpha = 0.4,bins = 100, label="kfsky")
# plt.hist(kma_pred_final,alpha = 0.4,bins = 100,label="kma")
# plt.hist(mogmog_pred_final,alpha = 0.4,bins = 100,label = "mogmog")
# plt.legend();

In [ ]:
# sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/sample_submission.csv")
# sub["score"] = (kfsky_pred_final + kma_pred_final + mogmog_pred_final)
# sub['score'] = sub['score'].rank(method='first')
# sub[['comment_id', 'score']].to_csv('submission.csv', index=False)
# sub.head()